In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
%load_ext tensorboard

In [2]:
'''
import glob
import pickle
files = glob.glob("../../datasets/ant-*.csv", recursive = True)
globa = pd.concat(map(pd.read_csv, files))
pickle.dump( globa, open( "ant_master.p", "wb" ) )
'''

'\nimport glob\nimport pickle\nfiles = glob.glob("../../datasets/ant-*.csv", recursive = True)\ngloba = pd.concat(map(pd.read_csv, files))\npickle.dump( globa, open( "ant_master.p", "wb" ) )\n'

In [3]:
ant_master = pickle.load( open( "jedit_master.p", "rb" ) )

In [4]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,jEdit,4,org.gjt.sp.jedit.gui.MacrosMenu,6,8,0,10,22,15,1,...,0.0,0,0.994206,0.333333,1,7,20.333333,5,2.1667,0
1,jEdit,4,org.gjt.sp.jedit.options.ModeOptionPane,12,6,0,7,39,28,3,...,1.0,2,0.983655,0.500000,1,3,45.416667,2,1.1667,1
2,jEdit,4,org.gjt.sp.jedit.gui.OptionsDialog,14,7,0,28,108,69,3,...,1.0,2,0.980480,0.174603,0,0,50.571429,9,2.1429,2
3,jEdit,4,org.gjt.sp.jedit.buffer.ExplicitFoldHandler,2,2,0,2,6,1,1,...,0.0,0,0.875000,0.625000,0,0,41.500000,5,2.5000,0
4,jEdit,4,org.gjt.sp.jedit.options.GeneralOptionPane,5,6,0,5,28,0,2,...,1.0,0,0.993994,0.500000,2,4,64.000000,3,1.6000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,jEdit,4.2,gnu.regexp.RETokenBackRef,3,2,0,4,8,0,1,...,1.0,0,0.777778,0.444444,1,2,24.000000,5,2.0000,0
363,jEdit,4.2,org.gjt.sp.jedit.pluginmgr.PluginList,5,1,0,11,35,0,4,...,0.0,0,0.000000,0.466667,0,0,40.000000,5,1.8000,0
364,jEdit,4.2,bsh.TargetError,10,4,0,16,25,23,11,...,0.0,0,0.764706,0.328571,1,2,12.300000,2,1.1000,0
365,jEdit,4.2,org.gjt.sp.jedit.ActionListHandler,11,2,0,5,28,31,1,...,1.0,1,0.565217,0.333333,1,3,20.000000,10,2.4545,0


In [5]:
test = ant_master[ant_master['version']==4.2]
test

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,jEdit,4.2,org.gjt.sp.jedit.gui.SelectLineRange,5,7,0,8,52,2,1,...,1.000000,1,0.993939,0.466667,0,0,72.400000,5,2.0000,0
1,jEdit,4.2,org.gjt.sp.jedit.options.GeneralOptionPane,7,6,0,6,36,3,2,...,1.000000,0,0.991111,0.500000,3,5,69.428571,6,1.8571,0
2,jEdit,4.2,org.objectweb.asm.CodeVisitor,17,1,0,5,17,136,4,...,0.000000,0,0.000000,0.394958,0,0,0.000000,1,1.0000,0
3,jEdit,4.2,org.gjt.sp.jedit.ActionContext,8,1,2,18,27,0,16,...,0.333333,0,0.000000,0.350000,0,0,18.000000,3,1.5000,0
4,jEdit,4.2,org.gjt.sp.jedit.textarea.Gutter,40,4,0,14,106,606,5,...,0.880000,3,0.943066,0.125000,3,17,25.875000,31,2.1000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,jEdit,4.2,gnu.regexp.RETokenBackRef,3,2,0,4,8,0,1,...,1.000000,0,0.777778,0.444444,1,2,24.000000,5,2.0000,0
363,jEdit,4.2,org.gjt.sp.jedit.pluginmgr.PluginList,5,1,0,11,35,0,4,...,0.000000,0,0.000000,0.466667,0,0,40.000000,5,1.8000,0
364,jEdit,4.2,bsh.TargetError,10,4,0,16,25,23,11,...,0.000000,0,0.764706,0.328571,1,2,12.300000,2,1.1000,0
365,jEdit,4.2,org.gjt.sp.jedit.ActionListHandler,11,2,0,5,28,31,1,...,1.000000,1,0.565217,0.333333,1,3,20.000000,10,2.4545,0


In [6]:
train = ant_master[ant_master['version']!=4.2]
train

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,jEdit,4,org.gjt.sp.jedit.gui.MacrosMenu,6,8,0,10,22,15,1,...,0.0,0,0.994206,0.333333,1,7,20.333333,5,2.1667,0
1,jEdit,4,org.gjt.sp.jedit.options.ModeOptionPane,12,6,0,7,39,28,3,...,1.0,2,0.983655,0.500000,1,3,45.416667,2,1.1667,1
2,jEdit,4,org.gjt.sp.jedit.gui.OptionsDialog,14,7,0,28,108,69,3,...,1.0,2,0.980480,0.174603,0,0,50.571429,9,2.1429,2
3,jEdit,4,org.gjt.sp.jedit.buffer.ExplicitFoldHandler,2,2,0,2,6,1,1,...,0.0,0,0.875000,0.625000,0,0,41.500000,5,2.5000,0
4,jEdit,4,org.gjt.sp.jedit.options.GeneralOptionPane,5,6,0,5,28,0,2,...,1.0,0,0.993994,0.500000,2,4,64.000000,3,1.6000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,jEdit,4.2,gnu.regexp.RETokenBackRef,3,2,0,4,8,0,1,...,1.0,0,0.777778,0.444444,1,2,24.000000,5,2.0000,0
363,jEdit,4.2,org.gjt.sp.jedit.pluginmgr.PluginList,5,1,0,11,35,0,4,...,0.0,0,0.000000,0.466667,0,0,40.000000,5,1.8000,0
364,jEdit,4.2,bsh.TargetError,10,4,0,16,25,23,11,...,0.0,0,0.764706,0.328571,1,2,12.300000,2,1.1000,0
365,jEdit,4.2,org.gjt.sp.jedit.ActionListHandler,11,2,0,5,28,31,1,...,1.0,1,0.565217,0.333333,1,3,20.000000,10,2.4545,0


In [7]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [8]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
0,6,8,0,10,22,15,1,10,4,2.000000,128,0.0,0,0.994206,0.333333,1,7,20.333333,5,2.1667
1,12,6,0,7,39,28,3,6,1,0.397727,573,1.0,2,0.983655,0.500000,1,3,45.416667,2,1.1667
2,14,7,0,28,108,69,3,28,8,0.869231,732,1.0,2,0.980480,0.174603,0,0,50.571429,9,2.1429
3,2,2,0,2,6,1,1,2,2,2.000000,85,0.0,0,0.875000,0.625000,0,0,41.500000,5,2.5000
4,5,6,0,5,28,0,2,4,1,0.708333,337,1.0,0,0.993994,0.500000,2,4,64.000000,3,1.6000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,3,2,0,4,8,0,1,3,0,0.500000,77,1.0,0,0.777778,0.444444,1,2,24.000000,5,2.0000
363,5,1,0,11,35,0,4,8,0,0.800000,210,0.0,0,0.000000,0.466667,0,0,40.000000,5,1.8000
364,10,4,0,16,25,23,11,6,10,0.500000,135,0.0,0,0.764706,0.328571,1,2,12.300000,2,1.1000
365,11,2,0,5,28,31,1,5,7,0.811111,240,1.0,1,0.565217,0.333333,1,3,20.000000,10,2.4545


In [9]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
0,5,7,0,8,52,2,1,8,4,0.500000,372,1.000000,1,0.993939,0.466667,0,0,72.400000,5,2.0000
1,7,6,0,6,36,3,2,6,1,0.788889,508,1.000000,0,0.991111,0.500000,3,5,69.428571,6,1.8571
2,17,1,0,5,17,136,4,1,17,2.000000,17,0.000000,0,0.000000,0.394958,0,0,0.000000,1,1.0000
3,8,1,2,18,27,0,16,3,8,0.523810,155,0.333333,0,0.000000,0.350000,0,0,18.000000,3,1.5000
4,40,4,0,14,106,606,5,14,36,0.929231,1100,0.880000,3,0.943066,0.125000,3,17,25.875000,31,2.1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,3,2,0,4,8,0,1,3,0,0.500000,77,1.000000,0,0.777778,0.444444,1,2,24.000000,5,2.0000
363,5,1,0,11,35,0,4,8,0,0.800000,210,0.000000,0,0.000000,0.466667,0,0,40.000000,5,1.8000
364,10,4,0,16,25,23,11,6,10,0.500000,135,0.000000,0,0.764706,0.328571,1,2,12.300000,2,1.1000
365,11,2,0,5,28,31,1,5,7,0.811111,240,1.000000,1,0.565217,0.333333,1,3,20.000000,10,2.4545


In [10]:
y_train = train['bug']
y_test = test['bug']

In [11]:
y_train

0      0
1      1
2      2
3      0
4      1
      ..
362    0
363    0
364    0
365    0
366    0
Name: bug, Length: 1257, dtype: int64

In [12]:
y_test

0      0
1      0
2      0
3      0
4      1
      ..
362    0
363    0
364    0
365    0
366    0
Name: bug, Length: 367, dtype: int64

In [13]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [14]:
x_train_np

array([[ 6.        ,  8.        ,  0.        , ..., 20.33333333,
         5.        ,  2.1667    ],
       [12.        ,  6.        ,  0.        , ..., 45.41666667,
         2.        ,  1.1667    ],
       [14.        ,  7.        ,  0.        , ..., 50.57142857,
         9.        ,  2.1429    ],
       ...,
       [10.        ,  4.        ,  0.        , ..., 12.3       ,
         2.        ,  1.1       ],
       [11.        ,  2.        ,  0.        , ..., 20.        ,
        10.        ,  2.4545    ],
       [ 4.        ,  2.        ,  0.        , ...,  8.        ,
         2.        ,  1.        ]])

In [15]:
x_test_np

array([[ 5.        ,  7.        ,  0.        , ..., 72.4       ,
         5.        ,  2.        ],
       [ 7.        ,  6.        ,  0.        , ..., 69.42857143,
         6.        ,  1.8571    ],
       [17.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       ...,
       [10.        ,  4.        ,  0.        , ..., 12.3       ,
         2.        ,  1.1       ],
       [11.        ,  2.        ,  0.        , ..., 20.        ,
        10.        ,  2.4545    ],
       [ 4.        ,  2.        ,  0.        , ...,  8.        ,
         2.        ,  1.        ]])

In [16]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [17]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [18]:
train_x

array([[[ 6.        ,  8.        ,  0.        , ..., 20.33333333,
          5.        ,  2.1667    ]],

       [[12.        ,  6.        ,  0.        , ..., 45.41666667,
          2.        ,  1.1667    ]],

       [[14.        ,  7.        ,  0.        , ..., 50.57142857,
          9.        ,  2.1429    ]],

       ...,

       [[10.        ,  4.        ,  0.        , ..., 12.3       ,
          2.        ,  1.1       ]],

       [[11.        ,  2.        ,  0.        , ..., 20.        ,
         10.        ,  2.4545    ]],

       [[ 4.        ,  2.        ,  0.        , ...,  8.        ,
          2.        ,  1.        ]]])

In [19]:
test_y

array([ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,
        0,  0,  1,  1,  1,  0,  0,  0,  0,  0,  0,  2,  0,  0,  1,  0,  0,
        0,  0,  3,  0,  2,  0,  0,  0,  0,  0,  0,  0,  4,  1,  0,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
        0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  6,  0,  0,  0,  2,  0,
        0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  4,  0,  1,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  9,
        0,  2,  0,  0,  0,  2,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  0,  0,  0,  0,  0,
        0,  0,  0,  1,  0

In [20]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [21]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, batch_size = 10, callbacks=[tensorboard_callback])

Epoch 1/100
126/126 [==============================] - 23s 24ms/step - loss: 6.3601 - mse: 6.3601 - mae: 0.8738 - root_mean_squared_error: 2.5219 - mean_squared_logarithmic_error: 0.3343
Epoch 2/100
126/126 [==============================] - 2s 18ms/step - loss: 5.5618 - mse: 5.5618 - mae: 0.8623 - root_mean_squared_error: 2.3583 - mean_squared_logarithmic_error: 0.2861
Epoch 3/100
126/126 [==============================] - 2s 18ms/step - loss: 5.1351 - mse: 5.1351 - mae: 0.8630 - root_mean_squared_error: 2.2661 - mean_squared_logarithmic_error: 0.2829
Epoch 4/100
126/126 [==============================] - 2s 19ms/step - loss: 5.0176 - mse: 5.0176 - mae: 0.8755 - root_mean_squared_error: 2.2400 - mean_squared_logarithmic_error: 0.2864
Epoch 5/100
126/126 [==============================] - 2s 18ms/step - loss: 4.8018 - mse: 4.8018 - mae: 0.8456 - root_mean_squared_error: 2.1913 - mean_squared_logarithmic_error: 0.2785
Epoch 6/100
126/126 [==============================] - 2s 19ms/step -

126/126 [==============================] - 2s 18ms/step - loss: 4.2952 - mse: 4.2952 - mae: 0.8639 - root_mean_squared_error: 2.0725 - mean_squared_logarithmic_error: 0.2704
Epoch 45/100
126/126 [==============================] - 2s 20ms/step - loss: 4.2096 - mse: 4.2096 - mae: 0.8889 - root_mean_squared_error: 2.0517 - mean_squared_logarithmic_error: 0.2789
Epoch 46/100
126/126 [==============================] - 2s 20ms/step - loss: 4.0679 - mse: 4.0679 - mae: 0.8328 - root_mean_squared_error: 2.0169 - mean_squared_logarithmic_error: 0.2617
Epoch 47/100
126/126 [==============================] - 2s 19ms/step - loss: 4.3506 - mse: 4.3506 - mae: 0.8595 - root_mean_squared_error: 2.0858 - mean_squared_logarithmic_error: 0.2782
Epoch 48/100
126/126 [==============================] - 2s 19ms/step - loss: 4.2361 - mse: 4.2361 - mae: 0.8819 - root_mean_squared_error: 2.0582 - mean_squared_logarithmic_error: 0.2804
Epoch 49/100
126/126 [==============================] - 2s 19ms/step - loss: 4

In [22]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6007 (pid 27262), started 1:33:57 ago. (Use '!kill 27262' to kill it.)

In [23]:
score = model.evaluate(test_x, test_y)
score

12/12 [==============================] - 2s 8ms/step - loss: 1.3208 - mse: 1.3208 - mae: 0.5442 - root_mean_squared_error: 1.1493 - mean_squared_logarithmic_error: 0.1752


[1.3207817077636719,
 1.3207817077636719,
 0.5442362427711487,
 1.1492526531219482,
 0.17518436908721924]